In [12]:
# Install all the packages listed in requirements.txt
import subprocess
import sys
def install_requirements():
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'])
        print("All packages installed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while installing packages: {e}")
install_requirements()

All packages installed successfully.


You should consider upgrading via the '/Users/raghulgopal/Desktop/MLOPs Dev/.venv/bin/python -m pip install --upgrade pip' command.


In [53]:
# Import all the packages to make them available for use
import os
from snowflake.snowpark import Session
from sklearn.feature_extraction import DictVectorizer
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode
from snowflake.ml.registry import Registry
import pandas as pd
from snowflake.snowpark.functions import col

In [14]:
# Connection Parameters to connect to snowflake
# Right now, I am using hardcoded values for the connection parameters.
# In a production environment, consider using environment variables or a secure vault to manage sensitive information.
# For example, you can use:
# connection_parameters = {
#     "account": os.getenv("SNOWFLAKE_ACCOUNT"),
#     "user": os.getenv("SNOWFLAKE_USER"),
#     "password": os.getenv("SNOWFLAKE_PASSWORD"),
#     "role": os.getenv("SNOWFLAKE_ROLE"),
#     "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
# }
# For example, you can use Secret Manager from AWS to store the secrtes, and retrieve them securely in your code.
# import boto3
# def get_secret(secret_name):
#     client = boto3.client('secretsmanager')
#     get_secret_value_response = client.get_secret_value(SecretId=secret_name)
#     secret = get_secret_value_response['SecretString']
#     return json.loads(secret)
connection_parameters = {
    "account": "XDHKYGF-ZJB97201",
    "user": "SERVICE_DEMO",
    "password": "Service@demo123",
    "role": "ML_ROLE",
    "warehouse": "COMPUTE_WH",
}
# Create a Snowflake session
session = Session.builder.configs(connection_parameters).create()
print("Snowflake session created successfully.")

Snowflake session created successfully.


#### Let's do the same feature engineerin step done in the No MLOps Level 0

In [37]:
def feature_engineering(df):
    # df is in the form of snowpark dataframe
    # For End to End MLOPs, we have to convert the snowpark dataframe to pandas dataframe
    # You can also do the development in snowpark dataframe itself, but for simplicity, we are converting it to pandas dataframe
    df = df.to_pandas()
    # our target variable is 'duration' which is the difference between 'lpep_dropoff_datetime' and 'lpep_pickup_datetime' in minutes
    df['DURATION'] = df['LPEP_DROPOFF_DATETIME'] - df['LPEP_PICKUP_DATETIME']
    df.DURATION = df.DURATION.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.DURATION >= 1) & (df.DURATION <= 60)]
    return df

read_data_from_snowflake = session.table("ML_DB.FEATURES.RAW_DATA") # Read data from the snowflake table - represented as a Snowpark dataframe
TRAINING_DATA = feature_engineering(read_data_from_snowflake)
TRAINING_DATA

,VENDORID,LPEP_PICKUP_DATETIME,LPEP_DROPOFF_DATETIME,PASSENGER_COUNT,TRIP_DISTANCE,RATECODEID,STORE_AND_FWD_FLAG,PULOCATIONID,DOLOCATIONID,PAYMENT_TYPE,...,MTA_TAX,TIP_AMOUNT,TOLLS_AMOUNT,IMPROVEMENT_SURCHARGE,TOTAL_AMOUNT,CONGESTION_SURCHARGE,EHAIL_FEE,TRIP_TYPE,CBD_CONGESTION_FEE,DURATION
0,2,2025-08-01 00:33:57,2025-08-01 00:58:10,1.0,2.44,1.0,N,70,82,2.0,...,0.5,0.00,0.0,1.0,24.40,0.00,NaN,1.0,0.00,24.216667
1,2,2025-08-01 00:01:00,2025-08-01 00:12:58,1.0,4.06,1.0,N,130,216,2.0,...,0.5,0.00,0.0,1.0,20.90,0.00,NaN,1.0,0.00,11.966667
2,2,2025-08-01 00:11:02,2025-08-01 00:16:06,1.0,2.09,1.0,N,74,263,1.0,...,0.5,3.05,0.0,1.0,18.30,2.75,NaN,1.0,0.00,5.066667
3,2,2025-08-01 00:44:09,2025-08-01 00:47:04,1.0,0.53,1.0,N,42,41,2.0,...,0.5,0.00,0.0,1.0,7.60,0.00,NaN,1.0,0.00,2.916667
4,2,2025-08-01 00:57:06,2025-08-01 01:05:32,1.0,2.28,1.0,N,75,239,1.0,...,0.5,3.47,0.0,1.0,20.82,2.75,NaN,1.0,0.00,8.433333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46301,2,2025-08-31 22:56:00,2025-08-31 23:16:00,NaN,4.12,NaN,None,49,224,NaN,...,0.5,4.21,0.0,1.0,32.27,NaN,NaN,NaN,0.75,20.000000
46302,6,2025-08-31 22:32:06,2025-08-31 22:51:08,NaN,3.89,NaN,None,70,95,NaN,...,0.5,0.00,0.0,0.3,16.00,NaN,NaN,NaN,0.00,19.033333
46303,2,2025-08-31 22:39:31,2025-08-31 23:00:00,NaN,4.89,NaN,None,112,224,NaN,...,0.5,3.56,0.0,1.0,33.19,NaN,NaN,NaN,0.75,20.483333
46304,2,2025-08-31 23:43:00,2025-09-01 00:03:00,NaN,8.37,NaN,None,74,32,NaN,...,0.5,0.00,0.0,1.0,30.31,NaN,NaN,NaN,0.00,20.000000


In [ ]:
#Create a feature store to track the engineered features
feature_store = FeatureStore(
    session = session,
    database = "ML_DB",
    name = "FEATURES",
    default_warehouse = "COMPUTE_WH",
    creation_mode = CreationMode.CREATE_IF_NOT_EXIST
)

In [30]:
# Entities are metadata about the feature inside feature store
try:
    # retrieve existing entity
    vendor_id_entity = feature_store.get_entity("VENDOR_ID_ENTITY")
    print("Entity 'VENDOR_ID_ENTITY' retrieved successfully.")
except:
    # create entity if it does not exist
    vendor_id_entity = Entity(
        name = "VENDOR_ID_ENTITY",
        join_keys = ["VENDORID"],
        desc = "Vendor ID entity representing the taxi service provider."
    )
    # register the entity
    feature_store.register_entity(vendor_id_entity)
    print("Entity 'VENDOR_ID_ENTITY' created and registered successfully.")

Entity 'VENDOR_ID_ENTITY' created and registered successfully.


In [63]:
FEATURES_DF = session.table("ML_DB.FEATURES.RAW_DATA").select(
        col("VENDORID"),
        col("PULOCATIONID"),
        col("DOLOCATIONID"),
        col("TRIP_DISTANCE")
    )
FEATURES_DF = pd.concat([FEATURES_DF.to_pandas(),TRAINING_DATA['DURATION']], ignore_index=True, axis=1)
FEATURES_DF.columns = ["VENDORID", "PULOCATIONID", "DOLOCATIONID", "TRIP_DISTANCE", "DURATION"]
# Conver the pandas dataframe to snowpark dataframe
session.use_database("ML_DB")
session.use_schema("FEATURES")
FEATURES_DF = session.create_dataframe(FEATURES_DF)
# Store it in a snowflake table
FEATURES_DF.write.save_as_table("ML_DB.FEATURES.ENGINEERED_FEATURES", mode="overwrite")

In [65]:
# Define and register feature view
FEATURES_DF = session.table("ML_DB.FEATURES.ENGINEERED_FEATURES")
feature_view = FeatureView(
    name = "TAXI_TRIP_FEATURES",
    entities = [vendor_id_entity],
    feature_df = FEATURES_DF,
    desc = "Feature view containing taxi trip features including trip distance and duration."
)
feature_view = feature_store.register_feature_view(feature_view, version = 'V1', overwrite=True)
print("Feature view 'TAXI_TRIP_FEATURES' created and registered successfully.")

Feature view 'TAXI_TRIP_FEATURES' created and registered successfully.


In [69]:
# Create a training dataset from the feature view
training_dataset = feature_store.generate_dataset(
    name = "TAXI_TRIP_TRAINING_DATASET",
    spine_df = FEATURES_DF.select(col("VENDORID")),
    features = [feature_view],
    version = 'V1'
)
print("Training dataset 'TAXI_TRIP_TRAINING_DATASET' created successfully.")

Training dataset 'TAXI_TRIP_TRAINING_DATASET' created successfully.


In [71]:
training_dataset_sp = training_dataset.read.to_snowpark_dataframe()
training_dataset_sp.show()

------------------------------------------------------------------------------------------
|"VENDORID"  |"PULOCATIONID"  |"DOLOCATIONID"  |"TRIP_DISTANCE"     |"DURATION"          |
------------------------------------------------------------------------------------------
|2           |43              |230             |3.0199999809265137  |26.899999618530273  |
|2           |43              |230             |3.0199999809265137  |26.899999618530273  |
|2           |43              |230             |3.0199999809265137  |26.899999618530273  |
|2           |43              |230             |3.0199999809265137  |26.899999618530273  |
|2           |43              |230             |3.0199999809265137  |26.899999618530273  |
|2           |43              |230             |3.0199999809265137  |26.899999618530273  |
|2           |43              |230             |3.0199999809265137  |26.899999618530273  |
|2           |43              |230             |3.0199999809265137  |26.899999618530273  |

In [ ]:
# We are good to go now for Dict vectorizer, and model training using scikit-learn
